## Word2vec model for GOBBYKID

In order to display values representing words similarities, we first need to create some models to train with respect to our corpora. Such models will be ablo to use <i>word embeddings</i> in order to retrieve similarities between the words of each corpus. This will be accomplished thanks to the mapping of such words into of feature-vectors.</br>
The aim of this section is to develop some different <i><u>word2vec</u></i> models, each with different parameters.</br>
At the end of the training session, we will qualitatively decide which one performs better in the definition of word similarities and save such model (obviously, the models will be two for each try: one for the female authors corpus and the other one for the male authors corpus). 

---------

### Setup

#### Install required libraries

To have an overview of the libraries used in this project, you can check the [README file](README.md).

Once followed the instructions provided in the above mentioned file, you can import the libraries, as well as the functions defined in the [normalizations functions file](normalization_functions.py).

In [7]:
from normalization_functions import *

---------

### Reading the corpora

Seen that the purpose of this project is the one defined in the [README file](README.md), we have obviously decided to train our model on the basis of the two corpora that we aim at analyzing.

In order to create a w2v model, we first need to create the two corpora for which we want to build the model, then we will store all the urls of the texts contained inside them into two different lists.</br>
Such lists will be passed as input to a function developed to store all the tokens of a corpus inside a list. During this step, texts will also need some preprocessing operation that will be performed by the same <span style="color:#89FC00"><i>list_builder</i></span> function, contained inside the [normalization functions file](normalization_functions.py), that will also create the two lists of which we were talking before.

In [8]:
f_directory = "assets/Raw corpora/F/"
m_directory = "assets/Raw corpora/M/"
f_corpus = create_corpus(f_directory)
m_corpus = create_corpus(m_directory)

In [9]:
# Female authors corpus
f_authors_texts = list()
for url in f_corpus.fileids():
    if url != '.DS_Store':
        f_authors_texts.append(f_directory+url)

# Male authors corpus
m_authors_texts = list()
for url in m_corpus.fileids():
    if url != '.DS_Store':
        m_authors_texts.append(m_directory+url)

These are the urls extracted:

In [10]:
print("URLS of female authors texts:", f_authors_texts, '\n')
print("URLS of male authors texts:", m_authors_texts, '\n')

URLS of female authors texts: ['assets/Raw corpora/F/1839_sinclair-holiday-house-a-series-of-tales.txt', 'assets/Raw corpora/F/1841_martineau-the-settlers-at-home.txt', 'assets/Raw corpora/F/1857_browne-grannys-wonderful-chair.txt', 'assets/Raw corpora/F/1857_tucker-the-rambles-of-a-rat.txt', 'assets/Raw corpora/F/1862_ewing-melchiors-dream-and-other-tales.txt', 'assets/Raw corpora/F/1869_ewing-mrs-overtheways-remembrances.txt', 'assets/Raw corpora/F/1869_ewing-the-land-of-lost-toys.txt', 'assets/Raw corpora/F/1870_ewing-the-brownies-and-other-tales.txt', 'assets/Raw corpora/F/1872_craik-the-adventure-of-a-brownie.txt', 'assets/Raw corpora/F/1872_de-la-ramee-a-dog-of-flanders.txt', 'assets/Raw corpora/F/1873_ewing-a-flat-iron-for-a-farthing.txt', 'assets/Raw corpora/F/1875_craik-the-little-lame-prince-and-his-traveling-cloack.txt', 'assets/Raw corpora/F/1876_ewing-jan-of-the-windmill.txt', 'assets/Raw corpora/F/1876_ewing-six-to-sixteen-a-story-for-girls.txt', 'assets/Raw corpora/F/187

Now, as we said above, we will go on by creating the two lists:

In [11]:
f_tokens = list_builder(f_authors_texts)

FileNotFoundError: [Errno 2] No such file or directory: 'Useful elements and texts/common_ws_list.txt'

In [ ]:
m_tokens = list_builder(m_authors_texts)

---------

### Models

Once we have the two lists, we need to initialize the two different models thanks to the Gensim library.</br>
Each model will be used for the respective corpus on which it has been trained.

To this end, we will create six groups of models and see which between these will perform better.</br>
The parameters will be different for each group, and now we will provide a brief overiview of what we want to do.</br>
In the word2vec model we may use 2 different methodologies to retrieve/produce word embeddings:
- <span style="color:#EE6352">CBOW (Continuous Bag Of Words)</span> - this first methodology operates by trying to solve a sort of "fake problem": given a context, the model tries to predict a target missing word. This is the base on which the model is trained in CBOW;
- <span style="color:#EE6352">Skip Grams</span> - the methodologies operates in an opposite way, in fact it tries to solve a similar problem but, this time, the input is the target word and the output will be its context.

These are two different ways to train a word2vec model.</br></br>
Together with these different ways to compute embeddings, we will try to change the <i>context-window size</i>, that is basically the amount of words that are part of the context we aim at analyzing (the size represents the amount of words on the left side and on the right side of the target word; e.g for a size of 3, the 3 words on the left and the 3 words on the right of a target word will be considered).</br>
The different window sizes are computed after reading the article [Dependency-Based Word Embeddings](https://levyomer.files.wordpress.com/2014/04/dependency-based-word-embeddings-acl-2014.pdf), which states that <i>larger windows tend to capture more topic/domain information, while smaller windows tend to capture more information about the word itself</i>.</br>
        To this end, we will produce three groups of models for each methodology, and these will have small window size, medium window size and large window size.

<span style="color:#F0A202"><i>PAY ATTENTION: In order to use the "workers" parameter defined inside each w2v model object, you need first to install <b>Cython</b>. Such parameter operates by deciding how many core of the CPU will be used for operations concerning the model. If Cython is not installed, the model will use by default one single core, therefore the operations will take way too much time to be performed.</i></span>

-------

#### CBOW Models

<b><span style="color:#EE6352">Small window</span></b>

The first operation to perform is the construction of a model. </br>
Inside the model, some parameters can be specified. For instance, we will work only with:
- <i>window</i>: defines the window size;
- <i>min_count</i>: defines the minnimum length of sentences to be considered;
- <i>workers</i>: defines the CPU cores that will compute calculations for the model (my CPU has 8 cores, if you have a CPU with less cores, please modify this value);
- <i>sg</i>: can assume the value of 1 or 0 (the default value), if its value is 1, then it will use skip grams to train the model, if the value is 0 it will train the model on a CBOW approach.

In [ ]:
f_sw_cbow_model = gensim.models.Word2Vec(
    window=3,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

In [ ]:
m_sw_cbow_model = gensim.models.Word2Vec(
    window=3,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

The next step consists in building <i>vocabularies</i>, that is a collection of all the single tokens for each one of our models.</br>
In order to build vocabularies we will use the <span style="color:#89FC00">build_vocab()</span> method, provided by Gensim.

In [ ]:
f_sw_cbow_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_sw_cbow_model.build_vocab(m_tokens, progress_per=1000)

-------

<b><span style="color:#EE6352">Medium window</span></b>

In [ ]:
f_mw_cbow_model = gensim.models.Word2Vec(
    window=6,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

In [ ]:
m_mw_cbow_model = gensim.models.Word2Vec(
    window=6,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

In [ ]:
f_mw_cbow_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_mw_cbow_model.build_vocab(m_tokens, progress_per=1000)

--------

<b><span style="color:#EE6352">Large window</span></b>

In [ ]:
f_lw_cbow_model = gensim.models.Word2Vec(
    window=9,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

In [ ]:
m_lw_cbow_model = gensim.models.Word2Vec(
    window=9,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=0
)

In [ ]:
f_lw_cbow_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_lw_cbow_model.build_vocab(m_tokens, progress_per=1000)

----------

#### Skip Gram Models

While CBOW model is better in capturing syntactic relationships, the Skip Gram model preforms better in extracting semantic relationships.</br>
For the aim of the w2v analysis on our project, we prefer to use Skip Gram models.

<b><span style="color:#EE6352">Small window</span></b>

In [ ]:
f_sw_skip_model = gensim.models.Word2Vec(
    window=3,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
m_sw_skip_model = gensim.models.Word2Vec(
    window=3,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
f_sw_skip_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_sw_skip_model.build_vocab(m_tokens, progress_per=1000)

-------

<b><span style="color:#EE6352">Medium window</span></b>

In [ ]:
f_mw_skip_model = gensim.models.Word2Vec(
    window=6,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
m_mw_skip_model = gensim.models.Word2Vec(
    window=6,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
f_mw_skip_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_mw_skip_model.build_vocab(m_tokens, progress_per=1000)

--------

<b><span style="color:#EE6352">Large window</span></b>

In [ ]:
f_lw_skip_model = gensim.models.Word2Vec(
    window=9,
    min_count=2, # This parameter do not consider sentences with "x" words or less, where x is the integer value specified
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
m_lw_skip_model = gensim.models.Word2Vec(
    window=9,
    min_count=2,
    workers=8, #This parameter is intended for CPU cores, if you have a CPU with less than 8 cores please modify the value
    sg=1
)

In [ ]:
f_lw_skip_model.build_vocab(f_tokens, progress_per=1000)

In [ ]:
m_lw_skip_model.build_vocab(m_tokens, progress_per=1000)

--------

### Train the models

The next step consists on training the different groups of models that we have built. We will do it thanks to the Gensim <span style="color:#89FC00">.train()</span> method.

<b><span style="color:#EE6352">Small window</span></b>

In [ ]:
f_sw_cbow_model.train(f_tokens, total_examples=f_sw_cbow_model.corpus_count, epochs=f_sw_cbow_model.epochs)
m_sw_cbow_model.train(m_tokens, total_examples=m_sw_cbow_model.corpus_count, epochs=m_sw_cbow_model.epochs)

(6225925, 7220150)

In [ ]:
f_sw_skip_model.train(f_tokens, total_examples=f_sw_skip_model.corpus_count, epochs=f_sw_skip_model.epochs)
m_sw_skip_model.train(m_tokens, total_examples=m_sw_skip_model.corpus_count, epochs=m_sw_skip_model.epochs)

(6225366, 7220150)

--------------

<b><span style="color:#EE6352">Medium window</span></b>

In [ ]:
f_mw_cbow_model.train(f_tokens, total_examples=f_mw_cbow_model.corpus_count, epochs=f_mw_cbow_model.epochs)
m_mw_cbow_model.train(m_tokens, total_examples=m_mw_cbow_model.corpus_count, epochs=m_mw_cbow_model.epochs)

(6225594, 7220150)

In [ ]:
f_mw_skip_model.train(f_tokens, total_examples=f_mw_skip_model.corpus_count, epochs=f_mw_skip_model.epochs)
m_mw_skip_model.train(m_tokens, total_examples=m_mw_skip_model.corpus_count, epochs=m_mw_skip_model.epochs)

(6225291, 7220150)

--------

<b><span style="color:#EE6352">Large window</span></b>

Maybe, this section provides the best output for our porject.

In [ ]:
f_lw_cbow_model.train(f_tokens, total_examples=f_lw_cbow_model.corpus_count, epochs=f_lw_cbow_model.epochs)
m_lw_cbow_model.train(m_tokens, total_examples=m_lw_cbow_model.corpus_count, epochs=m_lw_cbow_model.epochs)

(6225955, 7220150)

In [ ]:
f_lw_skip_model.train(f_tokens, total_examples=f_lw_skip_model.corpus_count, epochs=f_lw_skip_model.epochs)
m_lw_skip_model.train(m_tokens, total_examples=m_lw_skip_model.corpus_count, epochs=m_lw_skip_model.epochs)

(6225471, 7220150)

--------

### Comparison between models

<b><span style="color:#EE6352">Small window</span></b>

In [ ]:
f_sw_cbow_model.wv.most_similar("lady")

[('girl', 0.9244422912597656),
 ('woman', 0.9203931093215942),
 ('boy', 0.9157501459121704),
 ('gentleman', 0.9148160219192505),
 ('fellow', 0.9064704179763794),
 ('lord', 0.8819812536239624),
 ('man', 0.866852343082428),
 ('joan', 0.8651301264762878),
 ('child', 0.8557811379432678),
 ("eliza's", 0.8529271483421326)]

In [ ]:
m_sw_cbow_model.wv.most_similar("lady")

[('woman', 0.9196522831916809),
 ('gentleman', 0.9139875173568726),
 ('girl', 0.9129568934440613),
 ('boy', 0.8716918230056763),
 ('child', 0.8662574887275696),
 ('bill', 0.8595135807991028),
 ('nurse', 0.8422639966011047),
 ('princess', 0.8321806788444519),
 ('soldier', 0.816895067691803),
 ('cat', 0.810930073261261)]

In [ ]:
f_sw_skip_model.wv.most_similar("lady")

[('woman', 0.7208845019340515),
 ('gentleman', 0.7192774415016174),
 ('girl', 0.7132947444915771),
 ('amabel', 0.7082114219665527),
 ("lady's", 0.699910581111908),
 ('squire', 0.686664342880249),
 ('maid', 0.6850476264953613),
 ('captain', 0.6845502257347107),
 ('owl', 0.6840946674346924),
 ('doctor', 0.6807634830474854)]

In [ ]:
m_sw_skip_model.wv.most_similar("lady")

[('woman', 0.8645579814910889),
 ('girl', 0.8310417532920837),
 ('gentleman', 0.7945993542671204),
 ('nurse', 0.7927919030189514),
 ('child', 0.7532076835632324),
 ('rosamond', 0.7507439851760864),
 ('maid', 0.7479321360588074),
 ('dot', 0.7403877377510071),
 ('brooke', 0.7380125522613525),
 ('waiter', 0.731119692325592)]

--------------

<b><span style="color:#EE6352">Medium window</span></b>

In [ ]:
f_mw_cbow_model.wv.most_similar("lady")

[('girl', 0.9276626110076904),
 ('woman', 0.9031352996826172),
 ('gentleman', 0.8978793025016785),
 ('boy', 0.8967921137809753),
 ('fellow', 0.8952675461769104),
 ('lord', 0.8848101496696472),
 ('child', 0.8608880639076233),
 ('grandmother', 0.841951847076416),
 ('fauntleroy', 0.8379821181297302),
 ('dinah', 0.8376433849334717)]

In [ ]:
m_mw_cbow_model.wv.most_similar("lady")

[('girl', 0.9374927878379822),
 ('woman', 0.8998954892158508),
 ('child', 0.8588231205940247),
 ('princess', 0.8497556447982788),
 ('gentleman', 0.8384674787521362),
 ('boy', 0.8249669075012207),
 ('tone', 0.8205220699310303),
 ('nurse', 0.8085982799530029),
 ('dot', 0.7912766933441162),
 ('dog', 0.7829529643058777)]

In [ ]:
f_mw_skip_model.wv.most_similar("lady")

[("lady's", 0.7128770351409912),
 ('girl', 0.710577666759491),
 ('miss', 0.7078231573104858),
 ('grizzel', 0.7021595239639282),
 ('woman', 0.7009577751159668),
 ('amabel', 0.7006787061691284),
 ('maid', 0.6953164935112),
 ('captain', 0.6911873817443848),
 ('harriet', 0.686039924621582),
 ('grandmother', 0.683522641658783)]

In [ ]:
m_mw_skip_model.wv.most_similar("lady")

[('gentleman', 0.8069913983345032),
 ('woman', 0.7995109558105469),
 ('nurse', 0.7624256610870361),
 ('girl', 0.7434813380241394),
 ('esclairmonde', 0.7308054566383362),
 ('mistress', 0.7243124842643738),
 ('maid', 0.710415244102478),
 ('housekeeper', 0.7061601281166077),
 ('ghost', 0.7058016061782837),
 ('widow', 0.6996207237243652)]

--------

<b><span style="color:#EE6352">Large window</span></b>

In [ ]:
f_lw_cbow_model.wv.most_similar("lady")

[('girl', 0.870230495929718),
 ('naturalist', 0.8588536977767944),
 ('elizabeth', 0.853698194026947),
 ('tst', 0.8477090001106262),
 ('woman', 0.8383657932281494),
 ('harriet', 0.8310311436653137),
 ('louisa', 0.8240106701850891),
 ('joan', 0.8218511343002319),
 ('lavander', 0.8120749592781067),
 ('grandmother', 0.8105645179748535)]

In [ ]:
m_lw_cbow_model.wv.most_similar("lady")

[('girl', 0.9338809847831726),
 ('woman', 0.8962649703025818),
 ('child', 0.8599965572357178),
 ('princess', 0.8583437204360962),
 ('nurse', 0.8409541249275208),
 ('gentleman', 0.8110456466674805),
 ('tone', 0.7932117581367493),
 ('carrier', 0.7916038632392883),
 ('grandmother', 0.7886186838150024),
 ('boy', 0.7795196771621704)]

In [ ]:
f_lw_skip_model.wv.most_similar("lady")

[('harriet', 0.7367071509361267),
 ('madam', 0.7333415746688843),
 ('grizzel', 0.7155624628067017),
 ("lady's", 0.7094168663024902),
 ('rockville', 0.7055415511131287),
 ('podmore', 0.6939573287963867),
 ('tabitha', 0.6934369206428528),
 ('elizabeth', 0.6897143721580505),
 ('maid', 0.6893003582954407),
 ('ladies', 0.6891810894012451)]

In [ ]:
m_lw_skip_model.wv.most_similar("lady")

[('woman', 0.7675343155860901),
 ('gentleman', 0.7524054050445557),
 ('girl', 0.7390072345733643),
 ('nurse', 0.7348098158836365),
 ('maid', 0.7244940996170044),
 ('mistress', 0.7027565240859985),
 ('princess', 0.696609377861023),
 ('larkins', 0.6914660930633545),
 ('esclairmonde', 0.6872259974479675),
 ('tilly', 0.6860928535461426)]